In [6]:
! pip install qiskit

In [7]:
! pip install qiskit-machine-learning

In [8]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from qiskit import *

# External imports
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
#from qiskit.algorithms.optimizers import SPSA

from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data

#from qiskit.utils import QuantumInstance
#from qiskit_machine_learning.kernels import QuantumKernel

import random
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [9]:
df = pd.read_csv('Thesis IBM Dataset.csv')
df.head()

,Date,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,27-06-2023,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,27-06-2023,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,27-06-2023,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,27-06-2023,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,27-06-2023,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0


In [10]:
df.drop(columns=['Date'], inplace=True)
#df.drop(columns=['Temperature(C)'], inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [11]:
df.dropna(inplace=True)
df

,Time,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,27,32,21,87,3.0,1003,Cloudy,Cloudy,0.0,1,0,0
1,09:00,27,32,22,86,9.0,1003,Cloudy,Cloudy,0.0,1,0,0
2,10:00,28,34,23,90,3.0,1003,Cloudy,Rain,1.0,1,0,0
3,11:00,28,33,30,85,9.0,1003,Isolated Thunderstroms,Rain,1.0,1,0,0
4,12:00,27,32,53,90,9.0,1003,Rain,Sunny,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,1014,Clear,Clear,0.0,0,1,0
4500,20:00,19,19,1,80,14.0,1015,Clear,Clear,0.0,0,1,0
4501,21:00,19,19,2,82,14.0,1015,Clear,Clear,0.0,0,1,0
4502,22:00,18,18,2,83,14.0,1015,Clear,Clear,0.0,0,1,0


In [12]:
df.drop(columns=['Time', 'State','Real State'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Cloudy,Sunny,Fog
0,27,32,21,87,3.0,1003,0.0,1,0,0
1,27,32,22,86,9.0,1003,0.0,1,0,0
2,28,34,23,90,3.0,1003,1.0,1,0,0
3,28,33,30,85,9.0,1003,1.0,1,0,0
4,27,32,53,90,9.0,1003,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0,0,1,0
4500,19,19,1,80,14.0,1015,0.0,0,1,0
4501,19,19,2,82,14.0,1015,0.0,0,1,0
4502,18,18,2,83,14.0,1015,0.0,0,1,0


In [13]:
df.drop(columns=[ 'Cloudy'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Sunny,Fog
0,27,32,21,87,3.0,1003,0.0,0,0
1,27,32,22,86,9.0,1003,0.0,0,0
2,28,34,23,90,3.0,1003,1.0,0,0
3,28,33,30,85,9.0,1003,1.0,0,0
4,27,32,53,90,9.0,1003,0.0,1,0
...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0,1,0
4500,19,19,1,80,14.0,1015,0.0,1,0
4501,19,19,2,82,14.0,1015,0.0,1,0
4502,18,18,2,83,14.0,1015,0.0,1,0


In [14]:
df.drop(columns=['Sunny'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain,Fog
0,27,32,21,87,3.0,1003,0.0,0
1,27,32,22,86,9.0,1003,0.0,0
2,28,34,23,90,3.0,1003,1.0,0
3,28,33,30,85,9.0,1003,1.0,0
4,27,32,53,90,9.0,1003,0.0,0
...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0,0
4500,19,19,1,80,14.0,1015,0.0,0
4501,19,19,2,82,14.0,1015,0.0,0
4502,18,18,2,83,14.0,1015,0.0,0


In [15]:
df.drop(columns=[ 'Fog'], inplace=True)
df

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain
0,27,32,21,87,3.0,1003,0.0
1,27,32,22,86,9.0,1003,0.0
2,28,34,23,90,3.0,1003,1.0
3,28,33,30,85,9.0,1003,1.0
4,27,32,53,90,9.0,1003,0.0
...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,1014,0.0
4500,19,19,1,80,14.0,1015,0.0
4501,19,19,2,82,14.0,1015,0.0
4502,18,18,2,83,14.0,1015,0.0


In [16]:
df.describe()

,Temperature(C),Feel likes(C),Chance of Prec.(%),Humidity(%),Visibity(km),Pressure(hPa),Rain
count,4503.000000,4503.000000,4503.000000,4503.000000,4503.000000,4503.000000,4503.000000
mean,26.287364,30.537419,24.204086,84.216744,8.966911,1010.021097,0.127915
std,4.355640,7.491821,27.919227,12.429870,3.512862,134.304567,0.334032
min,11.000000,13.000000,1.000000,43.000000,1.000000,993.000000,0.000000
25%,24.000000,24.000000,3.000000,77.000000,6.000000,1004.000000,0.000000
50%,27.000000,32.000000,13.000000,88.000000,9.000000,1007.000000,0.000000
75%,29.000000,35.000000,37.000000,94.000000,12.000000,1013.000000,0.000000
max,40.000000,64.000000,100.000000,100.000000,16.000000,10013.000000,1.000000


In [17]:
def create_training_and_testing_data_sub(df_, DATA_SIZE_):
    data_size = DATA_SIZE_

    indices = np.random.choice(df1.index, size=data_size, replace=False)
    subset_df = df1.loc[indices]

    print(indices)

    # get the values from the last column of the subset and store them in y_subset
    y_subset = subset_df['Rain'].values

    # get the values from all other columns of the subset and store them in X_subset
    x_subset = subset_df.iloc[:, :-1].values

    # spltting the dataset into train and test set
    x_train, x_test, y_train, y_test = train_test_split(x_subset, y_subset, test_size = 0.25, random_state = 31)

    return x_train, x_test, y_train, y_test

In [19]:
NUM_OF_EXPERIMENTS = int(input("Number of experiments to run: "))
DATA_SIZE = int(input("Size of dataset: "))
NUM_MAX_ITER = int(input("Number of max iterations: "))
data = []

for current_iter in range(NUM_OF_EXPERIMENTS):

    X_train, X_test, y_train, y_test = create_training_and_testing_data_sub(df, DATA_SIZE)

    data.append([X_train, X_test, y_train, y_test])

Number of experiments to run:  100
Size of dataset:  200
Number of max iterations:  1


Traceback (most recent call last):
  Cell In[19], line 8
    X_train, X_test, y_train, y_test = create_training_and_testing_data_sub(df, DATA_SIZE)
  Cell In[17], line 4 in create_training_and_testing_data_sub
    indices = np.random.choice(df1.index, size=data_size, replace=False)
NameError: name 'df1' is not defined

Use %tb to get the full traceback.


In [ ]:
def find_classical_accuracy(x_train_, x_test_, y_train_, y_test_):

    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train_, y_train_)
    labels_pred = svc_clf.predict(x_test_)

    svc_accuracy = accuracy_score(y_test_, labels_pred)
    svc_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
    svc_f1_score = f1_score(y_test_, labels_pred)

    return svc_accuracy, svc_score, svc_f1_score

In [ ]:
def find_classical_accuracy_dt(x_train_, x_test_, y_train_, y_test_):
    clf = DecisionTreeClassifier(random_state=42)

    # Train the classifier on the training data
    clf.fit(x_train_, y_train)

    # Make predictions on the testing data
    labels_pred = clf.predict(x_test_)

    svc_accuracy = accuracy_score(y_test_, labels_pred)
    svc_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
    svc_f1_score = f1_score(y_test_, labels_pred)

    return svc_accuracy, svc_score, svc_f1_score

In [ ]:
def find_classical_accuracy_rf(x_train_, x_test_, y_train_, y_test_):
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the classifier on the training data
    rfc.fit(X_train, y_train)


    # Make predictions on the testing data
    labels_pred = rfc.predict(x_test_)

    svc_accuracy = accuracy_score(y_test_, labels_pred)
    svc_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_pred)
    svc_f1_score = f1_score(y_test_, labels_pred)

    return svc_accuracy, svc_score, svc_f1_score

In [ ]:
svc_acc = []
svc_bacc = []
svc_f1_score = []
total_time_elapsed = 0

for current_iter in range(NUM_OF_EXPERIMENTS):

    print(f"Current experiment no: {current_iter+1}")

    X_train, X_test, y_train, y_test = data[current_iter][0], data[current_iter][1], data[current_iter][2], data[current_iter][3]

    accuracy_svm, balanced_accuracy_svm, classical_f1_score = find_classical_accuracy(X_train, X_test, y_train, y_test)
    svc_acc.append(accuracy_svm)
    svc_bacc.append(balanced_accuracy_svm)
    svc_f1_score.append(classical_f1_score)

    print(f"Accuracy, Balanced Accuracy and F1 Score Classical SVM: {accuracy_svm}, {balanced_accuracy_svm}, {classical_f1_score}")

    print(f"-------------------------------------------------------------------------------------------------------------------")

In [ ]:
print(svc_acc)
print("For Classical SVM:")
print("Mean Accuracy:       ", np.array(svc_acc).mean())
print("Standard deviation:  ", np.array(svc_acc).std())
print("Minimum Accuracy:    ", np.array(svc_acc).min())
print("Maximum Accuracy:    ", np.array(svc_acc).max())
svc_avg = np.array(svc_acc).mean()
svc_min = np.array(svc_acc).min()
svc_max = np.array(svc_acc).max()

In [ ]:
plt.hist(np.array(svc_acc))
plt.title('Accuracy Distribution for Classical SVM')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
print(svc_bacc)
print("For Classical SVM:")
print("Mean Balanced Accuracy:       ", np.array(svc_bacc).mean())
print("Standard deviation:  ", np.array(svc_bacc).std())
print("Minimum Balanced Accuracy:    ", np.array(svc_bacc).min())
print("Maximum Balanced Accuracy:    ", np.array(svc_bacc).max())
svc_avg = np.array(svc_bacc).mean()
svc_min = np.array(svc_bacc).min()
svc_max = np.array(svc_bacc).max()

In [ ]:
plt.hist(np.array(svc_bacc))
plt.title('Balanced Accuracy Distribution for Classical SVM')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
from sklearn.tree import DecisionTreeClassifier

svc_acc_dt = []
svc_bacc_dt = []
svc_f1_score_dt = []

for current_iter in range(NUM_OF_EXPERIMENTS):

    print(f"Current experiment no: {current_iter+1}")

    X_train, X_test, y_train, y_test = data[current_iter][0], data[current_iter][1], data[current_iter][2], data[current_iter][3]

    accuracy_svm, balanced_accuracy_svm, classical_f1_score = find_classical_accuracy_dt(X_train, X_test, y_train, y_test)
    svc_acc_dt.append(accuracy_svm)
    svc_bacc_dt.append(balanced_accuracy_svm)
    svc_f1_score_dt.append(classical_f1_score)

    print(f"Accuracy, Balanced Accuracy and F1 Score Classical SVM: {accuracy_svm}, {balanced_accuracy_svm}, {classical_f1_score}")

    print(f"-----------------------------------------------------------------------------------------------------------")

In [ ]:
print(svc_acc_dt)
print("For Classical Decision Tree:")
print("Mean Accuracy:       ", np.array(svc_acc_dt).mean())
print("Standard deviation:  ", np.array(svc_acc_dt).std())
print("Minimum Accuracy:    ", np.array(svc_acc_dt).min())
print("Maximum Accuracy:    ", np.array(svc_acc_dt).max())
svc_avg = np.array(svc_acc_dt).mean()
svc_min = np.array(svc_acc_dt).min()
svc_max = np.array(svc_acc_dt).max()

In [ ]:
plt.hist(np.array(svc_acc_dt))
plt.title('Accuracy Distribution for Classical Decision Tree')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
plt.hist(np.array(svc_bacc_dt))
plt.title('Balanced Accuracy Distribution for Classical Decision Tree')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
from sklearn.ensemble import RandomForestClassifier

svc_acc_rf = []
svc_bacc_rf = []
svc_f1_score_rf = []
total_time_elapsed = 0

for current_iter in range(NUM_OF_EXPERIMENTS):

    print(f"Current experiment no: {current_iter+1}")

    X_train, X_test, y_train, y_test = data[current_iter][0], data[current_iter][1], data[current_iter][2], data[current_iter][3]

    accuracy_svm, balanced_accuracy_svm, classical_f1_score = find_classical_accuracy_rf(X_train, X_test, y_train, y_test)
    svc_acc_rf.append(accuracy_svm)
    svc_bacc_rf.append(balanced_accuracy_svm)
    svc_f1_score_rf.append(classical_f1_score)

    print(f"Accuracy, Balanced Accuracy and F1 Score Classical SVM: {accuracy_svm}, {balanced_accuracy_svm}, {classical_f1_score}")

    print(f"----------------------------------------------------------------")

In [ ]:
print(svc_acc_rf)
print("For Classical SVM:")
print("Mean Accuracy:       ", np.array(svc_acc_rf).mean())
print("Standard deviation:  ", np.array(svc_acc_rf).std())
print("Minimum Accuracy:    ", np.array(svc_acc_rf).min())
print("Maximum Accuracy:    ", np.array(svc_acc_rf).max())
svc_avg = np.array(svc_acc_rf).mean()
svc_min = np.array(svc_acc_rf).min()
svc_max = np.array(svc_acc_rf).max()

In [ ]:
plt.hist(np.array(svc_acc_rf))
plt.title('Accuracy Distribution for Classical Random Forest')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
print(svc_bacc_rf)
print("For Classical Random Forest:")
print("Mean Balanced Accuracy:       ", np.array(svc_bacc_rf).mean())
print("Standard deviation:  ", np.array(svc_bacc_rf).std())
print("Minimum Balanced Accuracy:    ", np.array(svc_bacc_rf).min())
print("Maximum Balanced Accuracy:    ", np.array(svc_bacc_rf).max())
svc_avg = np.array(svc_bacc_rf).mean()
svc_min = np.array(svc_bacc_rf).min()
svc_max = np.array(svc_bacc_rf).max()

In [ ]:
plt.hist(np.array(svc_bacc_rf))
plt.title('Balanced Accuracy Distribution for Random Forest')
plt.xlabel('Accuracy')
plt.ylabel('Frequency');

In [ ]:
def find_closest_to_mean(arr):
    mean = sum(arr) / len(arr)  # Calculate mean of the list

    # Calculate the differences between each element and the mean, and store them in a tuple with their index
    diff_indices = [(abs(x - mean), i) for i, x in enumerate(arr)]

    # Sort the tuple by the difference values
    sorted_diff_indices = sorted(diff_indices)

    # Extract the indices of the two closest values
    index1 = sorted_diff_indices[0][1]
    index2 = sorted_diff_indices[1][1]
    return [index1, index2]

In [ ]:
def find_closest_to_max(arr):
    max_val = max(arr)  # Calculate max of the list

    # Calculate the differences between each element and the max, and store them in a tuple with their index
    diff_indices = [(abs(x - max_val), i) for i, x in enumerate(arr)]

    # Sort the tuple by the difference values
    sorted_diff_indices = sorted(diff_indices)

    # Extract the indices of the two closest values
    index1 = sorted_diff_indices[0][1]
    index2 = sorted_diff_indices[1][1]
    return [index1, index2]

In [ ]:
def find_closest_to_min(arr):
    min_val = min(arr)  # Calculate min of the list

    # Calculate the differences between each element and the min, and store them in a tuple with their index
    diff_indices = [(abs(x - min_val), i) for i, x in enumerate(arr)]

    # Sort the tuple by the difference values
    sorted_diff_indices = sorted(diff_indices)

    # Extract the indices of the two closest values
    index1 = sorted_diff_indices[0][1]
    index2 = sorted_diff_indices[1][1]
    return [index1, index2]

In [ ]:
fm_list = []
names_fm = []
def create_feature_maps(features_):
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Z'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Z')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['ZZ'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('ZZ')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['YY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('YY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Z', 'ZZ'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Z ZZ')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Y', 'YY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Y YY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['X', 'YY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('X YY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Z', 'YY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Z YY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Y', 'XY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Y XY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['X', 'XY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('X XY')
    feature_map = PauliFeatureMap(features_, reps=2, paulis=['Z', 'XY'], entanglement="linear")
    fm_list.append(feature_map)
    names_fm.append('Z XY')


In [ ]:
create_feature_maps(7)

In [ ]:
def train_and_fit_model(x_train_, x_test_, y_train_, y_test_, fm_):

    start = time.time()

    # Building Kernel
    dse_feature_map = fm_

    dse_backend = QuantumInstance(
        Aer.get_backend('qasm_simulator'), shots=1024, seed_simulator=12345, seed_transpiler=12345
    )

    dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
    qsvc = QSVC(quantum_kernel=dse_kernel)
    qsvc.fit(x_train_, y_train_)

    # Predict the labels
    labels_test = qsvc.predict(x_test_)

    accuracy_test = metrics.accuracy_score(y_true=y_test_, y_pred=labels_test)
    ba_score = metrics.balanced_accuracy_score(y_true=y_test_, y_pred=labels_test)
    q_f1_score = f1_score(y_test_, labels_test)

    elapsed = time.time() - start

    return accuracy_test, ba_score, q_f1_score, elapsed

In [ ]:
! pip install qiskit-aer

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.utils import QuantumInstance

In [ ]:
print("For Mean Performance: ")
indices_mean = find_closest_to_mean(svc_bacc)
print(indices_mean)

q_ba, q_a, q_f1 = [], [], []
svc_ba, svc_a, svc_f1 = [], [], []
dt_ba, dt_a, dt_f1 = [], [], []
rf_ba, rf_a, rf_f1 = [], [], []

for fm in fm_list:
    qba, qa, qf = [], [], []
    for idx in indices_mean:
        X_train, X_test, y_train, y_test = data[idx]
        acc, bacc, f1, elapsed = train_and_fit_model(X_train, X_test, y_train, y_test, fm)
        print(f"QSVM Accuracy, Balanced Accuracy, F1 Score, Elapsed Time: {acc}, {bacc}, {f1}, {elapsed}")
        qba.append(bacc)
        qa.append(acc)
        qf.append(f1)
    q_ba.append(qba)
    q_a.append(qa)
    q_f1.append(qf)
    svc_ba.append(svc_bacc[idx])
    svc_a.append(svc_acc[idx])
    svc_f1.append(svc_f1_score[idx])
    dt_ba.append(svc_bacc_dt[idx])
    dt_a.append(svc_acc_dt[idx])
    dt_f1.append(svc_f1_score_dt[idx])
    rf_ba.append(svc_bacc_rf[idx])
    rf_a.append(svc_acc_rf[idx])
    rf_f1.append(svc_f1_score_rf[idx])

print("\n")
print(sum(svc_ba))
print(len(svc_ba))
print(svc_ba)
print(f"Averge Balanced Accuracy for SVM: {sum(svc_ba)/len(svc_ba)}")
print(f"Average Accuracy for SVM: {sum(svc_a)/len(svc_a)}")
print(f"Average F1 Score for SVM: {sum(svc_f1)/len(svc_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Decision Tree: {sum(dt_ba)/len(dt_ba)}")
print(f"Average Accuracy for Pauli Decision Tree: {sum(dt_a)/len(dt_a)}")
print(f"Average F1 Score for Pauli Decision Tree: {sum(dt_f1)/len(dt_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Random Forest: {sum(rf_ba)/len(rf_ba)}")
print(f"Average Accuracy for Pauli Random Forest: {sum(rf_a)/len(rf_a)}")
print(f"Average F1 Score for Pauli Random Forest: {sum(rf_f1)/len(rf_f1)}")
print("\n");
for i in range(len(fm_list)):
    print(f"Averge Balanced Accuracy for Pauli {names_fm[i]}: {sum(q_ba[i])/len(q_ba[i])}")
    print(f"Average Accuracy for Pauli {names_fm[i]}: {sum(q_a[i])/len(q_a[i])}")
    print(f"Average F1 Score for Pauli {names_fm[i]}: {sum(q_f1[i])/len(q_f1[i])}")
    print("\n")

In [ ]:
print('Oishik')

In [ ]:
print("For Min Performance ")
indices_min = find_closest_to_min(svc_bacc)
print(indices_min)

q_ba, q_a, q_f1 = [], [], []
svc_ba, svc_a, svc_f1 = [], [], []
dt_ba, dt_a, dt_f1 = [], [], []
rf_ba, rf_a, rf_f1 = [], [], []

for fm in fm_list:
    qba, qa, qf = [], [], []
    for idx in indices_min:
        X_train, X_test, y_train, y_test = data[idx]
        acc, bacc, f1, elapsed = train_and_fit_model(X_train, X_test, y_train, y_test, fm)
        print(f"Accuracy, Balanced Accuracy, F1 Score, Elapsed Time: {acc}, {bacc}, {f1}, {elapsed}")
        qba.append(bacc)
        qa.append(acc)
        qf.append(f1)
    q_ba.append(qba)
    q_a.append(qa)
    q_f1.append(qf)
    svc_ba.append(svc_bacc[idx])
    svc_a.append(svc_acc[idx])
    svc_f1.append(svc_f1_score[idx])
    dt_ba.append(svc_bacc_dt[idx])
    dt_a.append(svc_acc_dt[idx])
    dt_f1.append(svc_f1_score_dt[idx])
    rf_ba.append(svc_bacc_rf[idx])
    rf_a.append(svc_acc_rf[idx])
    rf_f1.append(svc_f1_score_rf[idx])

print("\n")
print(f"Averge Balanced Accuracy for SVM: {sum(svc_ba)/len(svc_ba)}")
print(f"Average Accuracy for SVM: {sum(svc_a)/len(svc_a)}")
print(f"Average F1 Score for SVM: {sum(svc_f1)/len(svc_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Decision Tree: {sum(dt_ba)/len(dt_ba)}")
print(f"Average Accuracy for Pauli Decision Tree: {sum(dt_a)/len(dt_a)}")
print(f"Average F1 Score for Pauli Decision Tree: {sum(dt_f1)/len(dt_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Random Forest: {sum(rf_ba)/len(rf_ba)}")
print(f"Average Accuracy for Pauli Random Forest: {sum(rf_a)/len(rf_a)}")
print(f"Average F1 Score for Pauli Random Forest: {sum(rf_f1)/len(rf_f1)}")
print("\n");
for i in range(len(fm_list)):
    print(f"Averge Balanced Accuracy for Pauli {names_fm[i]}: {sum(q_ba[i])/len(q_ba[i])}")
    print(f"Average Accuracy for Pauli {names_fm[i]}: {sum(q_a[i])/len(q_a[i])}")
    print(f"Average F1 Score for Pauli {names_fm[i]}: {sum(q_f1[i])/len(q_f1[i])}")
    print("\n")

In [ ]:
print("For Max Performance ")
indices_max = find_closest_to_max(svc_bacc)
print(indices_max)

q_ba, q_a, q_f1 = [], [], []
svc_ba, svc_a, svc_f1 = [], [], []
dt_ba, dt_a, dt_f1 = [], [], []
rf_ba, rf_a, rf_f1 = [], [], []

for fm in fm_list:
    qba, qa, qf = [], [], []
    for idx in indices_max:
        X_train, X_test, y_train, y_test = data[idx]
        acc, bacc, f1, elapsed = train_and_fit_model(X_train, X_test, y_train, y_test, fm)
        print(f"Accuracy, Balanced Accuracy, F1 Score, Elapsed Time: {acc}, {bacc}, {f1}, {elapsed}")
        qba.append(bacc)
        qa.append(acc)
        qf.append(f1)
    q_ba.append(qba)
    q_a.append(qa)
    q_f1.append(qf)
    svc_ba.append(svc_bacc[idx])
    svc_a.append(svc_acc[idx])
    svc_f1.append(svc_f1_score[idx])
    dt_ba.append(svc_bacc_dt[idx])
    dt_a.append(svc_acc_dt[idx])
    dt_f1.append(svc_f1_score_dt[idx])
    rf_ba.append(svc_bacc_rf[idx])
    rf_a.append(svc_acc_rf[idx])
    rf_f1.append(svc_f1_score_rf[idx])

print("\n")
print(f"Averge Balanced Accuracy for SVM: {sum(svc_ba)/len(svc_ba)}")
print(f"Average Accuracy for SVM: {sum(svc_a)/len(svc_a)}")
print(f"Average F1 Score for SVM: {sum(svc_f1)/len(svc_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Decision Tree: {sum(dt_ba)/len(dt_ba)}")
print(f"Average Accuracy for Pauli Decision Tree: {sum(dt_a)/len(dt_a)}")
print(f"Average F1 Score for Pauli Decision Tree: {sum(dt_f1)/len(dt_f1)}")
print("\n");
print(f"Averge Balanced Accuracy for Random Forest: {sum(rf_ba)/len(rf_ba)}")
print(f"Average Accuracy for Pauli Random Forest: {sum(rf_a)/len(rf_a)}")
print(f"Average F1 Score for Pauli Random Forest: {sum(rf_f1)/len(rf_f1)}")
print("\n");
for i in range(len(fm_list)):
    print(f"Averge Balanced Accuracy for Pauli {names_fm[i]}: {sum(q_ba[i])/len(q_ba[i])}")
    print(f"Average Accuracy for Pauli {names_fm[i]}: {sum(q_a[i])/len(q_a[i])}")
    print(f"Average F1 Score for Pauli {names_fm[i]}: {sum(q_f1[i])/len(q_f1[i])}")
    print("\n")

In [ ]:
from qiskit import *

In [ ]:
IBMQ.load_account()

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

In [ ]:
print("Using 3 features Price, Open, High: ")
sz = []
svm_acc = []
qsvm_acc = []
for data_size in range(20, 80, 10):
    print("for size", data_size)
    sz.append(data_size)
    x = df.iloc[:data_size, 0:-1]
    y = df.iloc[:data_size, -1]
    # spltting the dataset into train and test set
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 31)
    from sklearn import svm
    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train, y_train)
    svc_score = svc_clf.score(x_test, y_test)
    print(f"svm classification score: {svc_score}")
    svm_acc.append(svc_score)
    import qiskit
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.svm import SVC
    from sklearn.cluster import SpectralClustering
    from sklearn.metrics import normalized_mutual_info_score

    from qiskit import BasicAer
    from qiskit.circuit.library import ZZFeatureMap
    from qiskit.utils import QuantumInstance, algorithm_globals
    from qiskit_machine_learning.algorithms import QSVC
    from qiskit_machine_learning.kernels import QuantumKernel
    from qiskit_machine_learning.datasets import ad_hoc_data

    seed = 12345
    algorithm_globals.random_seed = seed
    # Building Kernel
    dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

    dse_backend = QuantumInstance(
        BasicAer.get_backend('qasm_simulator'), shots=1024, seed_simulator=seed, seed_transpiler=seed
    )

    dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
    dse_svc = SVC(kernel=dse_kernel.evaluate)
    dse_svc.fit(x_train, y_train)
    dse_score = dse_svc.score(x_test, y_test)

    print(f"qsvm classification test score: {dse_score}")
    qsvm_acc.append(dse_score)

In [ ]:
import random
total = 50
for cnt in range(total):
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 200-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        from qiskit_machine_learning.kernels import QuantumKernel
        from qiskit_machine_learning.datasets import ad_hoc_data

        seed = 12345
        algorithm_globals.random_seed = seed
        # Building Kernel
        dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

        dse_backend = QuantumInstance(
            BasicAer.get_backend('qasm_simulator'), shots=1024, seed_simulator=seed, seed_transpiler=seed
        )

        dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
        dse_svc = SVC(kernel=dse_kernel.evaluate)
        dse_svc.fit(x_train, y_train)
        dse_score = dse_svc.score(x_test, y_test)

        print(f"qsvm classification test score: {dse_score}")
        qsvm_acc[idx] = qsvm_acc[idx] + dse_score

In [ ]:
print(svm_acc)
print(qsvm_acc)

In [ ]:
print(svm_acc)
print(qsvm_acc)

In [ ]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/32, '\t\t', qsvm_acc[idx]/32)

In [ ]:
sz = [20, 30, 40, 50, 60]
svm_acc = [0.5, 0.8333333333333334, 0.75, 0.7, 0.5]
qsvm_acc = [.25, .5, .25, .6, 0.4166666666666667]
import random
total = 32
for cnt in range(total):
    print("current iteration: ", cnt)
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 200-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        from qiskit_machine_learning.kernels import QuantumKernel
        from qiskit_machine_learning.datasets import ad_hoc_data

        seed = 12345
        algorithm_globals.random_seed = seed
        # Building Kernel
        dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

        dse_backend = QuantumInstance(
            BasicAer.get_backend('qasm_simulator'), shots=1024, seed_simulator=seed, seed_transpiler=seed
        )

        dse_kernel = QuantumKernel(feature_map=dse_feature_map, quantum_instance=dse_backend)
        dse_svc = SVC(kernel=dse_kernel.evaluate)
        dse_svc.fit(x_train, y_train)
        dse_score = dse_svc.score(x_test, y_test)

        print(f"qsvm classification test score: {dse_score}")
        qsvm_acc[idx] = qsvm_acc[idx] + dse_score
    print("=============================================")

In [ ]:
print(svm_acc, qsvm_acc)

In [ ]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/32, '\t\t', qsvm_acc[idx]/32)